## Import Module

In [1]:
import random
import os
import pickle
import numpy as np
import itertools
from datetime import datetime
import pandas as pd
from pathlib import Path
from typing import Dict, List
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import math

# Data Loader

### using args

In [2]:
class MetaStockDataset(torch.utils.data.Dataset):
    def __init__(self,args):
        super(MetaStockDataset,self).__init__()
        self.args = args
        self.no_cuda = args.no_cuda
        self.data_path =  args.data_path
#         self.date_format = "%Y-%m-%d" 
        self.verbose = args.verbose
        self.fnames = [fname for fname in os.listdir(self.data_path) if
              os.path.isfile(os.path.join(self.data_path, fname))]
        self.data_EOD = []
        
        for index, fname in enumerate(self.fnames):
            # print(fname)
            single_EOD = np.genfromtxt(
                os.path.join(self.data_path, fname), dtype=float, delimiter=',',
                skip_header=False
            )
            # print('data shape:', single_EOD.shape)
            self.data_EOD.append(single_EOD)
        
        self.trading_dates = np.genfromtxt(
            os.path.join(self.data_path, '..', 'trading_dates.csv'), dtype=str,
            delimiter=',', skip_header=False)
        self.windows= [5, 10, 15, 20] 
        random.seed(self.args.seed)
        self.train_stock_num = args.train_stock_num
        self.test_stock_num = len(self.fnames) - self.train_stock_num
        self.train_date = args.train_date
        self.test_date = args.test_date
        self.train_stock = []
        
        """
        dataset ref: https://arxiv.org/abs/1810.09936
        In this meta learning setting, we have 3 meta-test and 1 meta-train
        vertical = stocks, horizontal = time
                train      |    test
           A               |
           B   meta-train  |   meta-test
           C               |      (1)
           ----------------|-------------
           D   meta-test   |   meta-test
           E     (2)       |      (3)

        meta-test (1) same stock, different time
        meta-test (2) different stock, same time
        meta-test (3) different stock, different time
        use `valid_date` to split the train / test set
        
        """
        def split_data(train_date, test_date):
            # train_date: start date for meta-train dataset
            # test_date: start date for meta-test dataset

            dates_index = {}
            for index, date in enumerate(self.trading_dates):
                dates_index[date] = index

            train_ind = dates_index[train_date]
            test_ind = dates_index[test_date]
            return train_ind, test_ind 

        self.train_ind, self.test_ind = split_data(self.train_date, self.test_date)
        if args.train == 'train':
            self.metasplit = ['train']

        elif args.train == 'test1':
            self.metasplit = ['test1']
            self.train_stock = args.train_stock
        elif args.train == 'test2':
            self.metasplit = ['test2']

        elif args.train == 'test3':
            self.metasplit = ['test3']

        assert args.train in ['train', 'test1','test2','test3']

    def task_formation(self,task_num):
        stock = []
        support = [[],[]]
        support_idx = [[],[]]
        support_stock = []
        query = [[],[]]
        query_idx =[[],[]]
        

        for i in range(task_num):

            if self.metasplit == ['train']:
                tick_ind = random.randint(0,self.train_stock_num)
                stock.append(tick_ind)
            elif self.metasplit == ['test1']:
                tick_ind = random.choice(self.train_stock)
                stock.append(tick_ind)

            elif self.metasplit == ['test2'] or self.metasplit == ['test3']:
                tick_ind = random.randint(self.train_stock_num, len(self.fnames)-1)
                stock.append(tick_ind)

            fname = self.fnames[tick_ind]
            single_EOD = np.genfromtxt(os.path.join(self.data_path, fname), dtype=float, delimiter=',',skip_header=False)

            def target_date_selection(window):
                if self.metasplit == ['train'] or self.metasplit == ['test2']:
                    target_date_ind = random.randint(self.train_ind, self.test_ind)
                if self.metasplit == ['test1'] or self.metasplit == ['test3']:
                    target_date_ind = random.randint(self.test_ind, len(self.trading_dates)-2)
                while target_date_ind <= window or abs(single_EOD[target_date_ind][-2]) < 1e-8 or \
                abs(single_EOD[target_date_ind+1][-2]) < 1e-8 or single_EOD[target_date_ind - window: target_date_ind, :].min() < -123320:
                    if self.metasplit == ['train'] or self.metasplit == ['test2']:
                        target_date_ind = random.randint(self.train_ind, self.test_ind)
                    if self.metasplit == ['test1'] or self.metasplit == ['test3']:
                        target_date_ind = random.randint(self.test_ind, len(self.trading_dates)-2)

                return target_date_ind

            for window in self.windows:
                target_date_ind = target_date_selection(window)
                x_support_idx = np.array(range(target_date_ind-window, target_date_ind))
                y_support_idx = target_date_ind
                support_idx[0].append(x_support_idx)
                support_idx[1].append(y_support_idx)
                support[0].append(single_EOD[x_support_idx, :11])
                support[1].append((single_EOD[y_support_idx][-2]+1)/2)

                x_query_idx = np.array(range(target_date_ind-window, target_date_ind+1))
                y_query_idx = target_date_ind+1
                query_idx[0].append(x_query_idx)
                query_idx[1].append(y_query_idx)
                query[0].append(single_EOD[x_query_idx, :11])
                query[1].append((single_EOD[y_query_idx][-2]+1)/2)


        self.train_stock = stock
        return stock, support_idx, support, query_idx, query      

                

            

## using config

In [81]:
## Config: Dict
# {'no_cuda': bool, 'data_path': str, 'verbose': int, 'seed':int, 'train_stock_num':int,
# 'train_date': str, 'test_date': str, 'train':str}

In [3]:
class MetaStockDataset(torch.utils.data.Dataset):
    def __init__(self,config):
        super(MetaStockDataset,self).__init__()
        self.config = config
        self.no_cuda = config['no_cuda']
        self.data_path =  config['data_path']
#         self.date_format = "%Y-%m-%d" 
        self.verbose = config['verbose']
        self.fnames = [fname for fname in os.listdir(self.data_path) if
              os.path.isfile(os.path.join(self.data_path, fname))]
        self.data_EOD = []
        
        for index, fname in enumerate(self.fnames):
            # print(fname)
            single_EOD = np.genfromtxt(
                os.path.join(self.data_path, fname), dtype=float, delimiter=',',
                skip_header=False
            )
            # print('data shape:', single_EOD.shape)
            self.data_EOD.append(single_EOD)
        
        self.trading_dates = np.genfromtxt(
            os.path.join(self.data_path, '..', 'trading_dates.csv'), dtype=str,
            delimiter=',', skip_header=False)
        self.windows= [5, 10, 15, 20] 
        random.seed(self.config['seed'])
        self.train_stock_num = config['train_stock_num']
        self.test_stock_num = len(self.fnames) - self.train_stock_num
        self.train_date = config['train_date']
        self.test_date = config['test_date']
        self.train_stock = []
        
        """
        dataset ref: https://arxiv.org/abs/1810.09936
        In this meta learning setting, we have 3 meta-test and 1 meta-train
        vertical = stocks, horizontal = time
                train      |    test
           A               |
           B   meta-train  |   meta-test
           C               |      (1)
           ----------------|-------------
           D   meta-test   |   meta-test
           E     (2)       |      (3)

        meta-test (1) same stock, different time
        meta-test (2) different stock, same time
        meta-test (3) different stock, different time
        use `valid_date` to split the train / test set
        
        """
        def split_data(train_date, test_date):
            # train_date: start date for meta-train dataset
            # test_date: start date for meta-test dataset

            dates_index = {}
            for index, date in enumerate(self.trading_dates):
                dates_index[date] = index

            train_ind = dates_index[train_date]
            test_ind = dates_index[test_date]
            return train_ind, test_ind 

        self.train_ind, self.test_ind = split_data(self.train_date, self.test_date)
        if config['train'] == 'train':
            self.metasplit = ['train']

        elif config['train'] == 'test1':
            self.metasplit = ['test1']
            self.train_stock = config['train_stock']
        elif config['train'] == 'test2':
            self.metasplit = ['test2']

        elif config['train'] == 'test3':
            self.metasplit = ['test3']

        assert config['train'] in ['train', 'test1','test2','test3']

    def task_formation(self,task_num):
        stock = []
        support = [[],[]]
        support_idx = [[],[]]
        support_stock = []
        query = [[],[]]
        query_idx =[[],[]]
        

        for i in range(task_num):

            if self.metasplit == ['train']:
                tick_ind = random.randint(0,self.train_stock_num)
                stock.append(tick_ind)
            elif self.metasplit == ['test1']:
                tick_ind = random.choice(self.train_stock)
                stock.append(tick_ind)

            elif self.metasplit == ['test2'] or self.metasplit == ['test3']:
                tick_ind = random.randint(self.train_stock_num, len(self.fnames)-1)
                stock.append(tick_ind)

            fname = self.fnames[tick_ind]
            single_EOD = np.genfromtxt(os.path.join(self.data_path, fname), dtype=float, delimiter=',',skip_header=False)

            def target_date_selection(window):
                if self.metasplit == ['train'] or self.metasplit == ['test2']:
                    target_date_ind = random.randint(self.train_ind, self.test_ind)
                if self.metasplit == ['test1'] or self.metasplit == ['test3']:
                    target_date_ind = random.randint(self.test_ind, len(self.trading_dates)-2)
                while target_date_ind <= window or abs(single_EOD[target_date_ind][-2]) < 1e-8 or \
                abs(single_EOD[target_date_ind+1][-2]) < 1e-8 or single_EOD[target_date_ind - window: target_date_ind, :].min() < -123320:
                    if self.metasplit == ['train'] or self.metasplit == ['test2']:
                        target_date_ind = random.randint(self.train_ind, self.test_ind)
                    if self.metasplit == ['test1'] or self.metasplit == ['test3']:
                        target_date_ind = random.randint(self.test_ind, len(self.trading_dates)-2)

                return target_date_ind

            for window in self.windows:
                target_date_ind = target_date_selection(window)
                x_support_idx = np.array(range(target_date_ind-window, target_date_ind))
                y_support_idx = target_date_ind
                support_idx[0].append(x_support_idx)
                support_idx[1].append(y_support_idx)
                support[0].append(single_EOD[x_support_idx, :11])
                support[1].append((single_EOD[y_support_idx][-2]+1)/2)

                x_query_idx = np.array(range(target_date_ind-window, target_date_ind+1))
                y_query_idx = target_date_ind+1
                query_idx[0].append(x_query_idx)
                query_idx[1].append(y_query_idx)
                query[0].append(single_EOD[x_query_idx, :11])
                query[1].append((single_EOD[y_query_idx][-2]+1)/2)


        self.train_stock = stock
        return stock, support_idx, support, query_idx, query      

                

            

## Prepare Dataset

### Train Set

In [27]:
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', default='/home/yjhwang/stock_prediction/data/stocknet-dataset/price/ourpped', type=str)
parser.add_argument('--no_cuda', default='False', type=bool)
parser.add_argument('--verbose', default=0, type=int)
parser.add_argument('--seed', default=300, type=int)
parser.add_argument('--train_stock_num', default=67, type=int)
parser.add_argument('--train_date', default='2014-01-02', type=str)
parser.add_argument('--test_date', default='2015-08-03', type=str)
parser.add_argument('--train', default='train', type=str)
parser.add_argument('--train_stock', default= [], type=list)
args = parser.parse_args(args=[])

In [24]:
config = {'no_cuda': False, 'data_path': '/home/yjhwang/stock_prediction/data/stocknet-dataset/price/ourpped', 'verbose': 0, 'seed': 300, 'train_stock_num':67,
'train_date': '2014-01-02', 'test_date': '2015-08-03', 'train':'train', 'train_stock': []}

In [25]:
train_data = MetaStockDataset(config)

In [28]:
args

Namespace(data_path='/home/yjhwang/stock_prediction/data/stocknet-dataset/price/ourpped', no_cuda=True, seed=300, test_date='2015-08-03', train='train', train_date='2014-01-02', train_stock=[], train_stock_num=67, verbose=0)

In [40]:
train_data = MetaStockDataset(args)

In [29]:
train_stock, train_support_idx,train_support, train_query_idx, train_query = train_data.task_formation(50)

In [34]:
len(train_support_idx[0])

200

In [35]:
torch.tensor(train_support[0][0])

tensor([[-1.7209e-01,  4.7941e-01, -1.7209e-01,  1.6494e+00,  1.6494e+00,
         -3.4420e-02,  8.0762e-01,  2.1955e+00,  2.9607e+00,  4.1711e+00,
          4.0746e+00],
        [-4.9168e-02,  7.9912e-01, -7.3764e-01, -1.2295e-02, -1.2298e-02,
         -5.6798e-01,  7.1675e-01,  1.9974e+00,  2.6856e+00,  3.7507e+00,
          4.0419e+00],
        [-4.0010e-01,  6.0616e-02, -1.0063e+00,  1.4015e+00,  1.4016e+00,
         -1.5762e+00, -6.4624e-01,  4.5180e-01,  9.8082e-01,  1.9912e+00,
          2.6495e+00],
        [-2.7060e-01,  8.4870e-01, -3.5670e-01, -1.4306e+00, -1.4307e+00,
         -2.0000e-06,  5.0061e-01,  1.5055e+00,  2.1402e+00,  3.0534e+00,
          4.1103e+00],
        [-8.7623e-01,  3.4076e-01, -8.8840e-01,  1.0701e+00,  1.0701e+00,
         -5.3790e-01, -6.9612e-01,  1.0872e-01,  9.1092e-01,  1.7388e+00,
          2.8652e+00]], dtype=torch.float64)

In [86]:
len(train_stock)

50

In [87]:
len(train_support[1])

200

In [88]:
# tick index & data sample index of the first task in support set
print("Tick index:", train_stock[0])
print("x_support index:", train_support_idx[0][0])
print("y_support index:", train_support_idx[1][0])

Tick index: 45
x_support index: [143 144 145 146 147]
y_support index: 148


In [32]:
#  x & y values in support set
print("x_support:\n", train_support[0][0])
print("y_support:\n", train_support[1][0])

x_support:
 [[-1.720950e-01  4.794100e-01 -1.720950e-01  1.649380e+00  1.649370e+00
  -3.442000e-02  8.076230e-01  2.195455e+00  2.960668e+00  4.171117e+00
   4.074581e+00]
 [-4.916800e-02  7.991170e-01 -7.376420e-01 -1.229500e-02 -1.229800e-02
  -5.679810e-01  7.167500e-01  1.997384e+00  2.685650e+00  3.750686e+00
   4.041893e+00]
 [-4.000980e-01  6.061600e-02 -1.006306e+00  1.401533e+00  1.401559e+00
  -1.576161e+00 -6.462440e-01  4.518010e-01  9.808200e-01  1.991246e+00
   2.649508e+00]
 [-2.706040e-01  8.487020e-01 -3.567050e-01 -1.430650e+00 -1.430667e+00
  -2.000000e-06  5.006070e-01  1.505527e+00  2.140215e+00  3.053376e+00
   4.110285e+00]
 [-8.762330e-01  3.407560e-01 -8.883970e-01  1.070105e+00  1.070102e+00
  -5.379040e-01 -6.961170e-01  1.087190e-01  9.109190e-01  1.738837e+00
   2.865202e+00]]
y_support:
 0.0


In [54]:
# tick index & data sample index of the first task in query set
print("Tick index:", train_stock[0])
print("x_support index:", train_query_idx[0][0])
print("y_support index:", train_query_idx[1][0])

Tick index: 45
x_support index: [143 144 145 146 147 148]
y_support index: 149


In [55]:
#  x & y values in query set
print("x_query:\n", train_query[0][0])
print("y_query:\n", train_query[1][0])

x_query:
 [[-1.720950e-01  4.794100e-01 -1.720950e-01  1.649380e+00  1.649370e+00
  -3.442000e-02  8.076230e-01  2.195455e+00  2.960668e+00  4.171117e+00
   4.074581e+00]
 [-4.916800e-02  7.991170e-01 -7.376420e-01 -1.229500e-02 -1.229800e-02
  -5.679810e-01  7.167500e-01  1.997384e+00  2.685650e+00  3.750686e+00
   4.041893e+00]
 [-4.000980e-01  6.061600e-02 -1.006306e+00  1.401533e+00  1.401559e+00
  -1.576161e+00 -6.462440e-01  4.518010e-01  9.808200e-01  1.991246e+00
   2.649508e+00]
 [-2.706040e-01  8.487020e-01 -3.567050e-01 -1.430650e+00 -1.430667e+00
  -2.000000e-06  5.006070e-01  1.505527e+00  2.140215e+00  3.053376e+00
   4.110285e+00]
 [-8.762330e-01  3.407560e-01 -8.883970e-01  1.070105e+00  1.070102e+00
  -5.379040e-01 -6.961170e-01  1.087190e-01  9.109190e-01  1.738837e+00
   2.865202e+00]
 [ 1.438761e+00  1.438761e+00 -2.126840e-01 -2.726053e+00 -2.726057e+00
   1.894164e+00  1.817844e+00  2.363738e+00  3.481800e+00  4.210689e+00
   5.333838e+00]]
y_query:
 -1.0


In [56]:
train_stock = train_data.train_stock

### Test1 set

In [57]:
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', default='/home/yjhwang/stock_prediction/data/stocknet-dataset/price/ourpped', type=str)
parser.add_argument('--no_cuda', default='False', type=bool)
parser.add_argument('--verbose', default=0, type=int)
parser.add_argument('--seed', default=200, type=int)
parser.add_argument('--train_stock_num', default=67, type=int)
parser.add_argument('--train_date', default='2014-01-02', type=str)
parser.add_argument('--test_date', default='2015-08-03', type=str)
parser.add_argument('--train', default='test1', type=str)
parser.add_argument('--train_stock', default= train_stock, type=list)
args = parser.parse_args(args=[])

In [58]:
test1_data = MetaStockDataset(args)

In [60]:
test1_stock, test1_support_idx, test1_support, test1_query_idx, test1_query = test1_data.task_formation(50)

In [61]:
# tick index & data sample index of the first task in support set
print("Tick index:", test1_stock[0])
print("x_support index:", test1_support_idx[0][0])
print("y_support index:", test1_support_idx[1][0])

Tick index: 45
x_support index: [606 607 608 609 610]
y_support index: 611


In [62]:
#  x & y values in support set
print("x_support:\n", test1_support[0][0])
print("y_support:\n", test1_support[1][0])

x_support:
 [[ 0.041038  0.30096  -0.670311 -0.381574 -0.381583  0.703148  0.604651
  -0.722299 -3.630643 -6.060191 -6.4259  ]
 [-0.517355  0.966654 -1.824365  0.478795  0.478789  0.487413  0.343096
  -0.890395 -3.279095 -5.957788 -6.669382]
 [ 0.124562  0.429063 -1.051906 -1.63376  -1.633758  1.749481  1.799307
   0.858131 -0.968166 -3.862699 -4.937946]
 [-0.596559  0.735293 -1.040511 -0.235291 -0.235291  1.07103   1.789675
   1.28838  -0.172035 -3.069371 -4.429803]
 [-1.091765  0.386951 -1.616914  0.388456  0.388455  0.398009  1.18021
   1.012529 -0.126452 -2.795468 -4.528746]]
y_support:
 1.0


In [63]:
# tick index & data sample index of the first task in query set
print("Tick index:", test1_stock[0])
print("x_query index:", test1_query_idx[0][0])
print("y_query index:", test1_query_idx[1][0])

Tick index: 45
x_query index: [606 607 608 609 610 611]
y_query index: 612


In [64]:
#  x & y values in query set
print("x_query:\n", test1_query[0][0])
print("y_query:\n", test1_query[1][0])

x_query:
 [[ 0.041038  0.30096  -0.670311 -0.381574 -0.381583  0.703148  0.604651
  -0.722299 -3.630643 -6.060191 -6.4259  ]
 [-0.517355  0.966654 -1.824365  0.478795  0.478789  0.487413  0.343096
  -0.890395 -3.279095 -5.957788 -6.669382]
 [ 0.124562  0.429063 -1.051906 -1.63376  -1.633758  1.749481  1.799307
   0.858131 -0.968166 -3.862699 -4.937946]
 [-0.596559  0.735293 -1.040511 -0.235291 -0.235291  1.07103   1.789675
   1.28838  -0.172035 -3.069371 -4.429803]
 [-1.091765  0.386951 -1.616914  0.388456  0.388455  0.398009  1.18021
   1.012529 -0.126452 -2.795468 -4.528746]
 [-1.616632  0.801516 -1.643797  1.727474  1.727472 -1.168319 -0.581441
  -0.451025 -1.34968  -3.672326 -5.787256]]
y_query:
 -1.0


### Test2 set

In [65]:
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', default='/home/yjhwang/stock_prediction/data/stocknet-dataset/price/ourpped', type=str)
parser.add_argument('--no_cuda', default='False', type=bool)
parser.add_argument('--verbose', default=0, type=int)
parser.add_argument('--seed', default=200, type=int)
parser.add_argument('--train_stock_num', default=67, type=int)
parser.add_argument('--train_date', default='2014-01-02', type=str)
parser.add_argument('--test_date', default='2015-08-03', type=str)
parser.add_argument('--train', default='test2', type=str)
parser.add_argument('--train_stock', default= [], type=list)
args = parser.parse_args(args=[])

In [66]:
test2_data = MetaStockDataset(args)

In [67]:
test2_stock, test2_support_idx, test2_support, test2_query_idx, test2_query = test2_data.task_formation(50)

In [68]:
# tick index & data sample index of the first task in support set
print("Tick index:", test2_stock[0])
print("x_support index:", test2_support_idx[0][0])
print("y_support index:", test2_support_idx[1][0])

Tick index: 68
x_support index: [519 520 521 522 523]
y_support index: 524


In [69]:
#  x & y values in support set
print("x_support:\n", test2_support[0][0])
print("y_support:\n", test2_support[1][0])

x_support:
 [[ 0.247991  0.7794   -0.023614 -0.458446 -0.458429  0.521958  0.123988
   0.062185  0.172404  0.512035  0.872293]
 [ 1.096149  1.096149  0.       -0.885687 -0.88569   1.050872  0.880492
   0.933312  0.997253  1.257708  1.669241]
 [-0.035773  0.286188 -0.441215 -0.083402 -0.083399  0.813254  0.945618
   0.984172  1.001659  1.182437  1.630484]
 [ 0.277714  0.760682 -0.024145 -1.240163 -1.240168  1.511712  2.104566
   2.088866  2.130519  2.274326  2.724378]
 [ 0.64904   2.043275 -0.480762  0.458823  0.458807  0.598576  1.460355
   1.512036  1.54749   1.70386   2.108989]]
y_support:
 -1.0


In [71]:
# tick index & data sample index of the first task in query set
print("Tick index:", test2_stock[0])
print("x_query index:", test2_query_idx[0][0])
print("y_query index:", test2_query_idx[1][0])

Tick index: 68
x_query index: [519 520 521 522 523 524]
y_query index: 525


In [72]:
#  x & y values in query set
print("x_query:\n", test2_query[0][0])
print("y_query:\n", test2_query[1][0])

x_query:
 [[ 0.247991  0.7794   -0.023614 -0.458446 -0.458429  0.521958  0.123988
   0.062185  0.172404  0.512035  0.872293]
 [ 1.096149  1.096149  0.       -0.885687 -0.88569   1.050872  0.880492
   0.933312  0.997253  1.257708  1.669241]
 [-0.035773  0.286188 -0.441215 -0.083402 -0.083399  0.813254  0.945618
   0.984172  1.001659  1.182437  1.630484]
 [ 0.277714  0.760682 -0.024145 -1.240163 -1.240168  1.511712  2.104566
   2.088866  2.130519  2.274326  2.724378]
 [ 0.64904   2.043275 -0.480762  0.458823  0.458807  0.598576  1.460355
   1.512036  1.54749   1.70386   2.108989]
 [ 1.104766  1.408275 -0.315651 -0.997589 -0.997585  1.051357  2.196193
   2.305051  2.41411   2.595608  2.950915]]
y_query:
 1.0


### Test3 set

In [73]:
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', default='/home/yjhwang/stock_prediction/data/stocknet-dataset/price/ourpped', type=str)
parser.add_argument('--no_cuda', default='False', type=bool)
parser.add_argument('--verbose', default=0, type=int)
parser.add_argument('--seed', default=150, type=int)
parser.add_argument('--train_stock_num', default=67, type=int)
parser.add_argument('--train_date', default='2014-01-02', type=str)
parser.add_argument('--test_date', default='2015-08-03', type=str)
parser.add_argument('--train', default='test3', type=str)
parser.add_argument('--train_stock', default= [], type=list)
args = parser.parse_args(args=[])

In [74]:
test3_data = MetaStockDataset(args)

In [75]:
test3_stock, test3_support_idx, test3_support, test3_query_idx, test3_query = test3_data.task_formation(50)

In [76]:
# tick index & data sample index of the first task in support set
print("Tick index:", test3_stock[0])
print("x_support index:", test3_support_idx[0][0])
print("y_support index:", test3_support_idx[1][0])

Tick index: 77
x_support index: [592 593 594 595 596]
y_support index: 597


In [77]:
#  x & y values in support set
print("x_support:\n", test3_support[0][0])
print("y_support:\n", test3_support[1][0])

x_support:
 [[ 0.311769  1.032732 -0.837882  0.509203  0.509199 -1.282148 -2.449333
  -2.081056 -1.449723 -1.155101 -0.519612]
 [-1.084225  0.367857 -1.742501  0.643028  0.643034 -1.23137  -2.58471
  -2.706685 -2.023238 -1.79439  -1.195875]
 [ 0.856362  1.245619 -0.875829 -0.522751 -0.522759 -0.128451 -1.911246
  -2.053973 -1.481116 -1.310233 -0.810299]
 [ 0.15601   0.409522 -0.780027 -0.194632 -0.194625  0.113105 -1.2812
  -1.78887  -1.259748 -1.063961 -0.746879]
 [-0.70299   0.976372 -0.742037 -0.136505 -0.136515  0.308541 -0.714694
  -1.546564 -1.201903 -0.963083 -0.758956]]
y_support:
 -1.0


In [79]:
# tick index & data sample index of the first task in query set
print("Tick index:", test2_stock[0])
print("x_query index:", test3_query_idx[0][0])
print("y_query index:", test3_query_idx[1][0])

Tick index: 68
x_query index: [592 593 594 595 596 597]
y_query index: 598


In [78]:
#  x & y values in query set
print("x_query:\n", test3_query[0][0])
print("y_query:\n", test3_query[1][0])

x_query:
 [[ 0.311769  1.032732 -0.837882  0.509203  0.509199 -1.282148 -2.449333
  -2.081056 -1.449723 -1.155101 -0.519612]
 [-1.084225  0.367857 -1.742501  0.643028  0.643034 -1.23137  -2.58471
  -2.706685 -2.023238 -1.79439  -1.195875]
 [ 0.856362  1.245619 -0.875829 -0.522751 -0.522759 -0.128451 -1.911246
  -2.053973 -1.481116 -1.310233 -0.810299]
 [ 0.15601   0.409522 -0.780027 -0.194632 -0.194625  0.113105 -1.2812
  -1.78887  -1.259748 -1.063961 -0.746879]
 [-0.70299   0.976372 -0.742037 -0.136505 -0.136515  0.308541 -0.714694
  -1.546564 -1.201903 -0.963083 -0.758956]
 [ 0.690061  0.985804 -0.670347 -0.95684  -0.956837  1.04101   0.46333
  -0.516559 -0.43178  -0.018924  0.185335]]
y_query:
 1.0


# Model

In [11]:
# attention lstm
class ALSTM(nn.Module):
    def __init__(self,input_size:int, hidden_size:int,  num_layers:int):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.lnorm = nn.LayerNorm(hidden_size)
        
    def forward(self, x:torch.tensor, rt_attn=False):
        #x: (W,L)
        o,(h,_) = self.lstm(x) # o: (B,W,H) / h: (1,B,H)
        print(o.shape, h.shape)
        score = torch.mm(o,h.permute(1,0)) # (B,W,H)*(B,H,1) = (B,W,1)
        tx_attn = F.softmax(score,dim=1) # (B,W)
        print(tx_attn.shape)
        context = torch.mm(torch.t(tx_attn),o) # (B,1,W)*(B,W,H) = (B,H)
        normed_context = self.lnorm(context) # (B,H)
        if rt_attn:
            return normed_context, tx_attn
        else:
            return normed_context, None
    
# generating latent code z 
class LatentEmbedding(nn.Module):
    def __init__(self,hidden_size:int):
        super().__init__()
        self.le = nn.Sequential(
            nn.Linear(hidden_size, 2*hidden_size, bias = False),
             nn.ReLU(),
             nn.Linear(2*hidden_size, 2*hidden_size, bias = False),
             nn.ReLU())

    def forward(self,x:torch.tensor):
        out = self.le(x) # 2*hidden_size
        return out
        
               
class MetaStock(nn.Module):
    # config : {'use_cuda': bool, 'input_size': int, 'hidden_size': int, 'num_layers': int, 'dropout': float, 'inner_l_rate_init': float,
    # 'finetuning_lr_init': float}
    def __init__(self,config):
        super().__init__()
        self._cuda = config['use_cuda']
        self.input_size = config['input_size']
        self.hidden_size = config['hidden_size']
        self.num_layers = config['num_layers']
        self.feature_transform = nn.Linear(self.input_size, self.hidden_size)
        self.encoder = ALSTM(self.hidden_size,self.hidden_size,self.num_layers)
        self.latent_embedding = LatentEmbedding(self.hidden_size)
        self.decoder = nn.Linear(self.hidden_size, 2*self.hidden_size, bias=False)
        self.dropout = nn.Dropout(p=config['dropout'])
        self.inner_l_rate = nn.Parameter(torch.FloatTensor([config['inner_l_rate_init']]))
        self.finetuning_lr = nn.Parameter(torch.FloatTensor([config['finetuning_lr_init']]))
        self.prob_layer = nn.LogSigmoid()
        
        
    def encode(self,inputs, rt_attn):
        # inputs -> [W,H]
        inputs = self.dropout(inputs)
        inputs = self.feature_transform(inputs)
        encoded, attn = self.encoder(inputs, rt_attn)
        h = self.latent_embedding(encoded)
        latents = self.sample(h, self.hidden_size)
        mean,var = h[:,:self.hidden_size], h[:,self.hidden_size:]
        kl_div = self.cal_kl_div(latents, mean, var)
        
        return encoded, attn, latents, kl_div
    
    def cal_kl_div(self,latents, mean, var):
        if self._cuda:
            return torch.mean(self.cal_log_prob(latents, mean, var) - self.cal_log_prob(latents, torch.zeros(mean.size()).cuda(), torch.ones(var.size()).cuda()))
        else:
            return torch.mean(self.cal_log_prob(latents, mean, var) - self.cal_log_prob(latents, torch.zeros(mean.size()), torch.ones(var.size())))
        
    def cal_log_prob(self,x,mean,var):
        eps = 1e-20
        log_unnormalized = - 0.5 * ((x - mean)/ (var+eps))**2
        log_normalization = torch.log(var+eps) + 0.5 * math.log(2*math.pi)

        return log_unnormalized - log_normalization      
    
    def sample(self, weights, size):
        mean, var = weights[:,:size], weights[:,size:]
        z = torch.normal(0.0,1.0, mean.size()).cuda()
        return mean+var*z
    
    def decode(self, latents):
        weights = self.decoder(latents)
        params = self.sample(weights,self.hidden_size)
        return params
    
    def predict(self,encoded,params):
        theta = params.view(-1, self.hidden_size, 1)
        scores = encoded.unsqueeze(dim=1).bmm(theta).squeeze()
        probs = torch.Tensor([self.prob_layer(scores)])
        return probs
    
    def cal_accuracy(self, log_probs, target):
        pred = (torch.exp(log_probs) >= 0.5).long()
        correct = pred.eq(target).sum()
        acc = correct / len(target)
        return acc

## Trainer

In [33]:
class ModelTrainer():
    def __init__(
        self,
        logging,
        log_dir,
        total_steps,
        total_val_steps,
        total_test_steps,
        n_inner_step,
        n_finetuning_step,
        outer_lr,
        verbose,
        beta,
        gamma,
        lambda1,
        lambda2,
        clip_value,
        load,
        model_dir,
        exp_name,
        save_best,
        save_checkpoint,
        use_cuda,
        n_tasks,
        train,
        valid_every_step,
        print_every_step,
        model_config,
        data_config):
        
        self.model = MetaStock(model_config)
        self.data = MetaStockDataset(data_config)
        self._verbose = verbose
        self._outer_lr = outer_lr
        self._total_steps = total_steps
        self._total_val_steps = total_val_steps
        self._total_test_steps = total_test_steps
        self._n_inner_step = n_inner_step
        self._n_finetuning_step = n_finetuning_step
        self._save_best = save_best
        self._save_checkpoint = save_checkpoint
        self._load_model = load
        self._use_cuda = use_cuda
        self._writer = SummaryWriter(log_dir)
        self._loss_fn = nn.NLLLoss()
        self._beta = beta # kl_weight
        self._gamma = gamma # encoder_penalty_weight
        self._lambda1 = lambda1 # l2-regularizer used in optimizer
        self._lambda2 = lambda2 # orthogonality penalty weight
        self._clip_value = clip_value,
        self._valid_every_step = valid_every_step
        self._print_every_step = print_every_step
        self._logging = logging
        self._n_tasks = n_tasks
        
        if train:
            self.model_dir = os.path.join(model_dir,exp_name)
            if not os.path.exists(self.model_dir):
                os.makedirs(self.model_dir)
        
        if self._save_best:
            self._best_acc = 0
        
        if self._use_cuda:
            self.model = self.model.cuda()
        
        for p in self.model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                
    def inner_loop(self,x_train,y_train):
        
        encoded, _, latents, kl_div = self.model.encode(x_train, rt_attn = False)
        latents_init = latents
        for i in range(self._n_inner_step):
            latents.retain_grad()
            params = self.model.decode(latents)
            params.retain_grad()
            probs = self.model.predict(encoded, params)
            print(probs)
            print(y_train)
            train_loss = self._loss_fn(probs,y_train).requires_grad_(True)
            print("train_loss:", train_loss)
            train_loss.backward(retain_graph=True)
            print(latents.grad_fn)
            latents = latents - self.model.inner_l_rate*latents.grad
        encoder_penalty = torch.mean((latents_init-latents)**2)
        return latents, kl_div, encoder_penalty
    
    def inner_finetuning(self, latents, x_train, y_train, x_val, y_val, verbose, step):
#         x_train, y_train = support_data[0], support_data[1]
#         x_val, y_val = query_data[0], query_data[1]
        params = self.model.decode(latents)
        params.retain_grad()
        train_loss = self._loss_fn(probs,y_train)
        train_acc = model.cal_accuracy(probs,y_train)
        
        # print info and logging
        if verbose and step % self._print_every_step == 0:
            print()
            print('(Meta-Train) [Step: %d/%d] Train Loss: %4.4f Train Accuracy: %4.4f Inner_Lr: %4.4f Finetuning_Lr: %4.4f ' \
                   %(step, self._total_steps, train_loss.item(), train_acc.item(), self.model.inner_l_rate, self.model.finetuning_lr))
        # logging
        if self._logging and step%self._print_every_step == 0:
            self.writer.add_scalar('Training Loss', train_loss.item(), step=step)
            self.writer.add_scalar('Training Accuracy', train_acc.item(), step=step)
            self.writer.add_scalar('Inner LR', float(model.inner_l_rate), step=step)
            self.writer.add_scalar('Finetuning LR', float(model.inner_l_rate), step=step)

        for j in range(self._n_finetuning_step):
            train_loss.backward(retain_graph=True)
            params = params - self.model.finetuning_lr * params.grad
            params.retain_grad()
            probs = model.predict(encoded, parameters)
            train_loss = self._loss_fn(probs, y_train)
            
       # validation  -> x_val이 encoder 부분 통과해야함   
    
        val_encoded, _ = model.encode(x_val, rt_attn=False)
        val_probs = model.predict(val_encoded, params)
        val_loss = self._loss_fn(val_probs,y_val)
        val_acc = model.cal_accuracy(val_probs, y_val) 
        
        return val_loss, val_acc
    
    def orthogonality(self,params):
        p2 = torch.mm(params,params.transpose(0,1))
        p_norm = torch.norm(params, dim=1, keepdim=True) + 1e-20
        correlation_matrix = w2 / torch.mm(p_norm, p_norm.transpose(0,1))
        I = torch.eye(correlation_matrix.size(0)).cuda()
        orthogonal_penalty = torch.mean((correlation_matrix-I)**2)
        return orthogonal_penalty
    
    def meta_train(self,x_train,y_train,x_val,y_val, step, train=True):
        # do task-train (inner loop)
        latents, kl_div, encoder_penalty = self.inner_loop(x_train,y_train)
        
        # do inner fine-tuning & task-validate (outer loop)
        val_loss, val_acc = self.inner_finetuning(latents, support_data,query_data, self._verbose and train, step)
        orthogonality_penalty = self.orthogonality(list(self.model.decoder.parameters())[0])
        
        # calculate loss (l2 reg implemented with optimizer)
        total_loss = val_loss + self._beta*kl_div + self._gamma*encoder_penalty + self._lambda2*orthogonality_penalty
        
        return total_loss, val_acc, kl_div, encoder_penalty, orthogonality_penalty
    
    def train(self):
        # different optim for lr and params (only l2 penalize on params)
        lr_list = ['inner_l_rate', 'finetuning_lr']
        params = [x[1] for x in list(filter(lambda k: k[0] not in lr_list, self.model.named_parameters()))]
        lr_params = [x[1] for x in list(filter(lambda k: k[0] in lr_list, self.model.named_parameters()))]
        optim = torch.optim.Adam(params, lr=self._outer_lr, weight_decay = self._lambda1)
        optim_lr = torch.optim.Adam(lr_params, lr=self._outer_lr)
        
        # update for (total steps) steps
        for step in range(self._total_steps):
            optim.zero_grad()
            optim_lr.zero_grad()
            #do training
            
            stock, support_idx, support, query_idx, query = self.data.task_formation(self._n_tasks)
            support_x, support_y = support[0], support[1]
            query_x, query_y = query[0], query[1]
            for i in range(4*self._n_tasks):
                
                x_train = torch.FloatTensor(support_x[i]).to(device='cuda:0') 
                y_train = torch.LongTensor([support_y[i]])
#                 y_train = torch.FloatTensor([support_y[i]]).to(device='cuda:0')
                x_val =  torch.FloatTensor(query_x[i]).to(device='cuda:0')
                y_val = torch.LongTensor([query_y[i]])
#                 y_val = torch.FloatTensor([query_y[i]]).to(device='cuda:0')
                print(x_train)
                print(y_train)
                print(x_val)
                print(y_val)
                total_loss, val_acc, encoder_penalty, orthogonality_penalty = self.meta_train(x_train,y_train,x_val,y_val, step, train=True)

                if self._verbose and step % self._print_every_step == 0:
                    print('(Meta-Valid) [Step: %d/%d] Total Loss: %4.4f Valid Accuracy: %4.4f'%(step, self._total_steps, val_loss.item(), val_acc.item()))
                    print('(Meta-Valid) [Step: %d/%d] KL: %4.4f Encoder Penalty: %4.4f Orthogonality Penalty: %4.4f'%(step, self._total_steps, kl_div, encoder_penalty, orthogonality_penalty))

                total_loss.backward()
                nn.utils.clip_grad_value_(self.model.parameters(),self._clip_value)
                nn.utils.clip_grad_norm_(self.model.parameters(), self._clip_value)
                optim.step()
                optim_lr.step()

                if step % self._valid_every_step == 1:
                    self.model.eval()
                    val_losses = []
                    val_accs = []

                    for val_step in range(self.total_val_steps):
                        val_loss, val_acc, _, _, _ = self.meta_train(x_train,y_train,x_val,y_val, step, train=False)
                        val_losses.append(val_loss.item())
                        val_accs.append(val_acc.item())

                    if self._save_checkpoint:
                        #save checkpoint                      
                        if not (self._save_best and sum(val_accs)/len(val_accs) < self._best_acc):
                            model_name = '%dk_%4.4f_%4.4f_model.pth' % (step//1000, sum(val_losses)/len(val_losses), sum(val_accs)/len(val_accs))
                            state = {'step': step, 'val_acc': sum(val_accs)/len(val_accs), 'state_dict': self.model.state_dict()}
                            if not os.path.exists(self.model_dir):
                                os.mkdir(self.model_dir)
                            torch.save(state, os.path.join(self.model_dir, model_name))

                    self.model.train()   
                    if self._verbose:
                        print()
                        print('=' * 50)
                        print('Meta Valid Loss: %4.4f \nMeta Valid Accuracy: %4.4f'%(sum(val_losses)/len(val_losses), sum(val_accs)/len(val_accs)))
                        print('=' * 50)
                        print()
                        if self._save_checkpoint:
                            print('Saving checkpoint %s...'%model_name)
                            print()
                        
    def test(self):
        

        #load state dict
        state_dict = torch.load(self._load_model)['state_dict']
        self.model.load_state_dict(state_dict)

        self.model.eval()
        test_losses = []
        test_accs = []
        
        for test_step in range(self._total_test_steps):
            stock, support_idx, support_data, query_idx, query_data = self.data.task_formation(self._n_tasks)
            test_loss, test_acc, _, _, _ = self.run_batch(suport_data, query_data, test_step, False)
            test_losses.append(test_loss.item())
            test_accs.append(test_acc.item())

        if self._verbose:
            print()
            print('=' * 50)
            print('Meta Test Loss: %4.4f Meta Test Accuracy: %4.4f'%(sum(test_losses)/len(test_losses), sum(test_accs)/len(test_accs)))
            print('=' * 50)
            print()


In [6]:
data_config =  {'no_cuda': False, 'data_path': '/home/yjhwang/stock_prediction/data/stocknet-dataset/price/ourpped', 'verbose': 0, 'seed': 300, 'train_stock_num':67,
'train_date': '2014-01-02', 'test_date': '2015-08-03', 'train':'train', 'train_stock': []}
model_config = {'use_cuda': True, 'input_size': 11, 'hidden_size': 32, 'num_layers': 1, 'dropout': 0.3, 'inner_l_rate_init': 1,
    'finetuning_lr_init': 0.001}

In [7]:
torch.cuda.is_available()

True

In [34]:
ModelTrainer(
        logging=True,
        log_dir="./log",
        total_steps=10,
        total_val_steps=10,
        total_test_steps=10,
        n_inner_step=10,
        n_finetuning_step=10,
        outer_lr=0.0001,
        verbose=True,
        beta=0.001,
        gamma=1.0e-9,
        lambda1=1.0e-8,
        lambda2=0.1,
        clip_value=0.1,
        load="./train_model/model.pth",
        model_dir="./train_model",
        exp_name="example",
        save_best=True,
        save_checkpoint=True,
        use_cuda=True,
        n_tasks=50,
        train=True,
        valid_every_step=2,
        print_every_step=2,
        model_config=model_config,
        data_config=data_config).train()

tensor([[-1.7210e-01,  4.7941e-01, -1.7210e-01,  1.6494e+00,  1.6494e+00,
         -3.4420e-02,  8.0762e-01,  2.1955e+00,  2.9607e+00,  4.1711e+00,
          4.0746e+00],
        [-4.9168e-02,  7.9912e-01, -7.3764e-01, -1.2295e-02, -1.2298e-02,
         -5.6798e-01,  7.1675e-01,  1.9974e+00,  2.6857e+00,  3.7507e+00,
          4.0419e+00],
        [-4.0010e-01,  6.0616e-02, -1.0063e+00,  1.4015e+00,  1.4016e+00,
         -1.5762e+00, -6.4624e-01,  4.5180e-01,  9.8082e-01,  1.9912e+00,
          2.6495e+00],
        [-2.7060e-01,  8.4870e-01, -3.5671e-01, -1.4306e+00, -1.4307e+00,
         -2.0000e-06,  5.0061e-01,  1.5055e+00,  2.1402e+00,  3.0534e+00,
          4.1103e+00],
        [-8.7623e-01,  3.4076e-01, -8.8840e-01,  1.0701e+00,  1.0701e+00,
         -5.3790e-01, -6.9612e-01,  1.0872e-01,  9.1092e-01,  1.7388e+00,
          2.8652e+00]], device='cuda:0')
tensor([0])
tensor([[-1.7210e-01,  4.7941e-01, -1.7210e-01,  1.6494e+00,  1.6494e+00,
         -3.4420e-02,  8.0762e-01,  2.195

/tmp/ipykernel_39111/1399484577.py:167: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y_train = torch.LongTensor([support_y[i]])
/tmp/ipykernel_39111/1399484577.py:170: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y_val = torch.LongTensor([query_y[i]])


AttributeError: 'NoneType' object has no attribute 'data'

In [16]:
torch.Tensor([-2]).requires_grad_(True)

tensor([-2.], requires_grad=True)